In [4]:
import json
import pandas as pd
from sentence_transformers.util import cos_sim  
from sentence_transformers import SentenceTransformer as SBert
from scipy.spatial.distance import cosine
from numpy import sqrt
import numpy as np
from geopy.distance import geodesic
from tqdm import trange
model = SBert('paraphrase-multilingual-MiniLM-L12-v2')
cols = ["id", "Location", "Name", "Address", "District", "RoadInfo", "Poi", "RoomInfo", "label"]

with open("../train_data/positive.json", "r", encoding="utf-8") as f:
    positive = json.load(f)
with open("../train_data/negative.json", "r", encoding="utf-8") as f:
    negative = json.load(f)
with open("../train_src/temp/pre_dict.json", "r", encoding="utf-8") as f:
    eDict = json.load(f)

In [5]:
def calcDist(pos1, pos2):
    if pos1[0] is None or pos2[0] is None:
        return None
    return geodesic((pos1[1], pos1[0]), (pos2[1], pos2[0])).kilometers

def calcSimi(lst1, lst2):
    lstCosine = []
    for i in range(len(lst1)):
        if lst1[i] == "" or lst2[i] == "":
            lstCosine.append(None)
        else:
            e1 = eDict[lst1[i]]
            e2 = eDict[lst2[i]]
            sim = 1 - cosine(e1, e2)
            lstCosine.append(sim)
    return lstCosine

def embData(data):
    dfLst = []
    for d in data:
        lst = [d["id"]]
        lst.append(calcDist(d["Location"][0], d["Location"][1]))
        lst1 = [d[attr][0] for attr in cols[2:8]]
        lst2 = [d[attr][1] for attr in cols[2:8]]
        lst = lst + calcSimi(lst1, lst2) + [d["label"]]
        dfLst.append(lst)
    df = pd.DataFrame(dfLst, columns=cols)
    df = df.fillna(value=np.nan)
    return df

In [6]:
dfPos = embData(positive)
dfNeg = embData(negative)
dfPos.to_csv("../train_data/positive.csv")
dfNeg.to_csv("../train_data/negative.csv")